In [26]:
import os
import numpy as np
import pandas as pd
from aeon.datasets import load_classification
import warnings

from utils.tools import create_directory
from utils.constants import datasets

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

warnings.filterwarnings("ignore")


# Problem Setting -----------------------------------------------------------------------------------------------------

# ----------------------------------------------------------------------------------------------------------------------
for problem in datasets[:1]:
    print("[Main] Problem: {}".format(problem))
    itr_result = [problem]
    
    for run in range(1):
        print("[Main] Run:", run + 1)
        # load data
        X_train, Y_train = load_classification(problem, split="train")
        X_test, Y_test = load_classification(problem, split="test")
        all_labels = np.hstack((Y_train, Y_test))
        x_train = X_train.reshape(X_train.shape[1], X_train.shape[0], X_train.shape[2])
        x_test = X_test.reshape(X_test.shape[1], X_test.shape[0], X_test.shape[2])
        print(X_train.shape)
        print(X_train[0])
        all_data = np.hstack((x_train, x_test))
        for i in  range(len(x_train)):
            # znorm
            std_ = x_train[0].std(axis=1, keepdims=True)
            std_[std_ == 0] = 1.0
            x_train[0] = (x_train[0] - x_train[0].mean(axis=1, keepdims=True)) / std_

            std_ = x_test[0].std(axis=1, keepdims=True)
            std_[std_ == 0] = 1.0
            x_test[0] = (x_test[0] - x_test[0].mean(axis=1, keepdims=True)) / std_
        X_train=x_train.reshape(X_train.shape)
        X_test=x_test.reshape(X_test.shape)
        print(X_train.shape)
        print(X_train[0])
      

[Main] Problem: ArticularyWordRecognition
[Main] Run: 1
(275, 9, 144)
[[ 0.87159  0.88042  0.88042 ... -0.95541 -0.64258 -0.64258]
 [ 0.7453   0.82313  0.82313 ...  2.1161   2.2079   2.2079 ]
 [ 1.1533   1.0887   1.0887  ...  1.045    1.286    1.286  ]
 ...
 [ 0.88441  0.85476  0.85476 ... -0.40933 -0.44269 -0.44269]
 [ 1.137    0.63292  0.63292 ... -2.5608  -2.3481  -2.3481 ]
 [ 1.2429   1.4795   1.4795  ... -1.0315  -1.0025  -1.0025 ]]
(275, 9, 144)
[[ 0.87463188  0.8834927   0.8834927  ... -0.95874366 -0.644822
  -0.644822  ]
 [ 0.74789647  0.8259979   0.8259979  ...  2.12347717  2.21559733
   2.21559733]
 [ 1.15732529  1.09249993  1.09249993 ...  1.04864748  1.29048822
   1.29048822]
 ...
 [ 0.88749736  0.8577438   0.8577438  ... -0.41076141 -0.44423793
  -0.44423793]
 [ 1.14097122  0.63513147  0.63513147 ... -2.56973786 -2.35629532
  -2.35629532]
 [ 1.24723946  1.48466562  1.48466562 ... -1.03510232 -1.00600106
  -1.00600106]]


In [24]:
import os
import numpy as np
import pandas as pd
from aeon.datasets import load_classification
import warnings

from utils.tools import create_directory
from utils.constants import datasets

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

warnings.filterwarnings("ignore")

def fit_classifier(all_labels, X_train, y_train, X_val=None, y_val=None, epochs=10, batch_size=16):
    nb_classes = len(np.unique(all_labels))

    input_shape = (X_train.shape[1], X_train.shape[2], X_train.shape[3])
    # Call Classifier ----------------------------------------------------------
    classifier = create_classifier(classifier_name, input_shape, nb_classes, verbose=True)
    # Train Classifier ----------------------------------------------------------
    if X_val is None:
        classifier.fit(X_train, y_train, None, None, epochs, batch_size)
    else:
        classifier.fit(X_train, y_train, X_val, y_val, epochs, batch_size)
    return classifier


def create_classifier(classifier_name, input_shape, nb_classes, verbose=False):
    if classifier_name == "Disjoint_CNN":
        from classifiers import Custom_Disjoint_CNN
        return Custom_Disjoint_CNN.Classifier_Disjoint_CNN(sub_output_directory, input_shape, nb_classes, verbose)


# Problem Setting -----------------------------------------------------------------------------------------------------
ALL_Results = pd.DataFrame()
ALL_Results_list = []
problem_index = 0
data_path = os.getcwd() + '/multivariate_ts/'
# Hyper-Parameter Setting ----------------------------------------------------------------------------------------------
classifier_name = "Disjoint_CNN"  # Choose the classifier name from aforementioned List
epochs = 500
Resample = 1  # Set to '1' for default Train and Test Sets, and '30' for running on all resampling
# ----------------------------------------------------------------------------------------------------------------------
for problem in datasets[:1]:
    # Load Data --------------------------------------------------------------------------------------------------------
    output_directory = os.getcwd() + '/Results_Custom_' + classifier_name + '/' + problem + '/'
    create_directory(output_directory)
    print("[Main] Problem: {}".format(problem))
    itr_result = [problem]
    
    for run in range(1):
        print("[Main] Run:", run + 1)
        # load data
        X_train, Y_train = load_classification(problem, split="train")
        X_test, Y_test = load_classification(problem, split="test")

        all_labels = np.hstack((Y_train, Y_test))
        from sklearn.preprocessing import StandardScaler
        scaler = StandardScaler()
        # all_data = scaler.fit_transform(all_data)
        x_train = X_train.reshape(X_train.shape[1], X_train.shape[0], X_train.shape[2])
        x_test = X_test.reshape(X_test.shape[1], X_test.shape[0], X_test.shape[2])
        all_data = np.hstack((x_train, x_test))
        for i in  range(len(x_train)):
            # znorm
            std_ = x_train[0].std(axis=1, keepdims=True)
            std_[std_ == 0] = 1.0
            x_train[0] = (x_train[0] - x_train[0].mean(axis=1, keepdims=True)) / std_

            std_ = x_test[0].std(axis=1, keepdims=True)
            std_[std_ == 0] = 1.0
            x_test[0] = (x_test[0] - x_test[0].mean(axis=1, keepdims=True)) / std_
        print(x_train[0])
      

[Main] Problem: ArticularyWordRecognition
[Main] Run: 1
[[ 0.87463188  0.8834927   0.8834927  ... -0.95874366 -0.644822
  -0.644822  ]
 [ 0.74789647  0.8259979   0.8259979  ...  2.12347717  2.21559733
   2.21559733]
 [ 1.15732529  1.09249993  1.09249993 ...  1.04864748  1.29048822
   1.29048822]
 ...
 [ 1.8403066   1.78320785  1.78320785 ...  0.21699602  0.24952927
   0.24952927]
 [-0.62624202 -0.76821576 -0.76821576 ...  0.35365572  0.57217565
   0.57217565]
 [ 0.06901797  0.02269286  0.02269286 ... -0.23322824 -0.27785344
  -0.27785344]]
